In [26]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format',  '{:.2f}'.format)

In [6]:
df = pd.read_csv("ds_salaries.csv", index_col='Unnamed: 0')
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...
602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


In [39]:
df.groupby("job_title")["work_year"].count()

job_title
3D Computer Vision Researcher                 1
AI Scientist                                  7
Analytics Engineer                            4
Applied Data Scientist                        5
Applied Machine Learning Scientist            4
BI Data Analyst                               6
Big Data Architect                            1
Big Data Engineer                             8
Business Data Analyst                         5
Cloud Data Engineer                           2
Computer Vision Engineer                      6
Computer Vision Software Engineer             3
Data Analyst                                 97
Data Analytics Engineer                       4
Data Analytics Lead                           1
Data Analytics Manager                        7
Data Architect                               11
Data Engineer                               132
Data Engineering Manager                      5
Data Science Consultant                       7
Data Science Engineer         

### Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.)

In [51]:
conditions = [
    (df.job_title.str.contains("Data Scientist")),
    (df.job_title.str.contains("Data Engineer")),
    (df.job_title.str.contains("Software Engineer"))
]
choices = ['DS', 'DE', 'Software Engenering']
df['direction'] = np.select(conditions, choices, default='etc')

In [52]:
df.groupby("direction")["work_year"].count()

direction
DE                     158
DS                     159
Software Engenering      3
etc                    287
Name: work_year, dtype: int64

### Какая средняя и медианная зарплата по группам вакансий?

In [53]:
df.groupby(['direction']).agg({'salary_in_usd': ['mean', 'median']})

salary_in_usd          
                             mean    median
direction                                  
DE                      115808.51 108912.00
DS                      115134.60 109000.00
Software Engenering     105248.67  95746.00
etc                     108867.30  98000.00

### Какая средняя и медианная зарплата по каждому региону?

In [54]:
df.groupby(['company_location']).agg({'salary_in_usd': ['mean', 'median']})

salary_in_usd          
                          mean    median
company_location                        
AE                   100000.00 115000.00
AS                    18053.00  18053.00
AT                    72920.75  69489.50
AU                   108042.67  87425.00
BE                    85699.00  85699.00
BR                    18602.67  18907.00
CA                    99823.73  81895.50
CH                    64114.00  64114.00
CL                    40038.00  40038.00
CN                    71665.50  71665.50
CO                    21844.00  21844.00
CZ                    50937.00  50937.00
DE                    81887.21  78015.00
DK                    54386.33  45896.00
DZ                   100000.00 100000.00
EE                    32974.00  32974.00
ES                    53060.14  48371.50
FR                    63970.67  56738.00
GB                    81583.04  78526.00
GR                    52293.09  49461.00
HN                    20000.00  20000.00
HR                    45618.00  45618.00
HU                    35735.00  35735.00
IE                    71444.00  71444.00
IL                   119059.00 119059.00
IN                    28581.75  22124.00
IQ                   100000.00 100000.00
IR                     4000.00   4000.00
IT                    36366.50  36366.50
JP                   114127.33  75682.00
KE                     9272.00   9272.00
LU                    43942.67  59102.00
MD                    18000.00  18000.00
MT                    28369.00  28369.00
MX                    32123.33  33511.00
MY                    40000.00  40000.00
NG                    30000.00  30000.00
NL                    54945.75  54021.00
NZ                   125000.00 125000.00
PK                    13333.33  12000.00
PL                    66082.50  41093.50
PT                    47793.75  54217.50
RO                    60000.00  60000.00
RU                   157500.00 157500.00
SG                    89294.00  89294.00
SI                    63831.00  63831.00
TR                    20096.67  20171.00
UA                    13400.00  13400.00
US                   144055.26 135000.00
VN                     4000.00   4000.00

### Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?

In [67]:
print('Самая оплачиваемая группа вакансий исходя из средних зарплат это ' + 
      df.groupby("direction")["salary_in_usd"].mean()
      .reset_index(name='mean')
      .sort_values('mean', ascending=False)
      .head(1)['direction'].tolist()[0])

Самая оплачиваемая группа вакансий исходя из средних зарплат это DE


### Какое процентное соотношение каждого региона по вакансиям от всех вакансий?

In [73]:
agg ={'company_location':['count']}
grouped=df.groupby('company_location').agg(agg).reset_index()
grouped.insert(2,'procent',' ',True)
for index, row in grouped.iterrows():
    grouped.at[index,'procent']= str(round(row[1]*100/len(df),2))+'%'
grouped.sort_values([('company_location','count')],ascending=False)

company_location       procent
                    count        
48               US   355  58.48%
18               GB    47   7.74%
6                CA    30   4.94%
12               DE    28   4.61%
25               IN    24   3.95%
17               FR    15   2.47%
16               ES    14   2.31%
19               GR    11   1.81%
29               JP     6   0.99%
40               PL     4   0.66%
41               PT     4   0.66%
37               NL     4   0.66%
2                AT     4   0.66%
34               MX     3   0.49%
31               LU     3   0.49%
46               TR     3   0.49%
39               PK     3   0.49%
0                AE     3   0.49%
3                AU     3   0.49%
5                BR     3   0.49%
13               DK     3   0.49%
9                CN     2   0.33%
11               CZ     2   0.33%
4                BE     2   0.33%
45               SI     2   0.33%
43               RU     2   0.33%
36               NG     2   0.33%
28               IT     2   0.33%
7                CH     2   0.33%
38               NZ     1   0.16%
8                CL     1   0.16%
15               EE     1   0.16%
44               SG     1   0.16%
47               UA     1   0.16%
42               RO     1   0.16%
10               CO     1   0.16%
35               MY     1   0.16%
14               DZ     1   0.16%
33               MT     1   0.16%
32               MD     1   0.16%
30               KE     1   0.16%
27               IR     1   0.16%
26               IQ     1   0.16%
1                AS     1   0.16%
24               IL     1   0.16%
23               IE     1   0.16%
22               HU     1   0.16%
21               HR     1   0.16%
20               HN     1   0.16%
49               VN     1   0.16%

### Какая корреляция уровня опыта от зарплаты?

In [78]:
conditions = [
    (df.experience_level == "EN"),
    (df.experience_level == "MI"),
    (df.experience_level == "SE"),
    (df.experience_level == "EX")
]
choices = [1, 2, 3, 4]
df['level'] = np.select(conditions, choices)
df['salary_in_usd'].corr(df['level'])

0.4842328861730616

### Сколько должностей в наборе данных?

In [79]:
print('Число должностей в датасете: ' + str(df['job_title'].nunique()))

Число должностей в датасете: 50


### Какие 10 наиболее часто встречающихся должностей?

In [91]:
df['job_title'].value_counts().reset_index().head(10)

,index,job_title
0,Data Scientist,143
1,Data Engineer,132
2,Data Analyst,97
3,Machine Learning Engineer,41
4,Research Scientist,16
5,Data Science Manager,12
6,Data Architect,11
7,Big Data Engineer,8
8,Machine Learning Scientist,8
9,Principal Data Scientist,7
